# Met Animals

## Part 1: Connect to the Database

In [1]:
# Import dependencies
from pymongo import MongoClient
import json
import requests
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the met database to a variable name
db = mongo["met"]

In [4]:
# assign the collection to a variable
artifacts = db["artifacts"]

## Part 2: Collect the Data

In [5]:
# Artifacts from department 5 and search string "cave"
# https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=5&q=cave
cave_ids = [
    313726,
    313724,
    312602,
    310364,
    318345,
    313256,
    310542,
    313240,
    310551,
    314557,
    310384,
    317901,
    317902,
    318730,
    310343,
    309557,
    307443,
    312268,
    778892,
    317619,
    312595,
    312915,
    317697,
    313325,
    310480,
    310644,
    309404,
    314142,
    317121,
    317120,
    316299,
    317430,
    317429,
    313151,
    317760,
    309960,
    313262,
    314217,
]

# Print the number of IDs
print(f"Number of IDs: {len(cave_ids)}")

# Initialize a list to store the returned documents
returned_data_list = []

# Create a loop to make API calls about each artifact we're interested in
for i in range(len(cave_ids)):
    # API URL
    url = "https://collectionapi.metmuseum.org/public/collection/v1/objects/" + str(
        cave_ids[i]
    )

    # Make the API request
    response = requests.get(url)

    # Extract the json result
    extract_json = response.json()

    # Add the returned json to the end of the list
    returned_data_list.append(extract_json)

# Print the length of the results to ensure it matches the cave_ids list length
print(f"Length of returned collection artifacts: {len(returned_data_list)}")

Number of IDs: 38
Length of returned collection artifacts: 38


Now we have our list of additional artifacts we want to add to our database, but we need to check that these artifacts are not already in our collection before we add them

## Part 3: Update the Database

In [6]:
# Choose just one item from our returned data and set it to a variable
item_to_add = returned_data_list[0]

# Pretty print item_to_add
pprint(item_to_add)

{'GalleryNumber': '',
 'accessionNumber': '1979.206.1537',
 'accessionYear': '1979',
 'additionalImages': [],
 'artistAlphaSort': '',
 'artistBeginDate': '',
 'artistDisplayBio': '',
 'artistDisplayName': '',
 'artistEndDate': '',
 'artistGender': '',
 'artistNationality': '',
 'artistPrefix': '',
 'artistRole': '',
 'artistSuffix': '',
 'artistULAN_URL': '',
 'artistWikidata_URL': '',
 'city': '',
 'classification': 'Wood-Musical Instruments',
 'constituents': None,
 'country': 'Australia',
 'county': '',
 'creditLine': 'The Michael C. Rockefeller Memorial Collection, Bequest of '
               'Nelson A. Rockefeller, 1979',
 'culture': 'Mulga Downs Cave',
 'department': 'The Michael C. Rockefeller Wing',
 'dimensions': 'H. 20 x W.  2 3/4 x D. 5/16 in. (50.8 x 7 x 0.8 cm)',
 'dynasty': '',
 'excavation': '',
 'geographyType': '',
 'isHighlight': False,
 'isPublicDomain': False,
 'isTimelineWork': False,
 'linkResource': '',
 'locale': '',
 'locus': '',
 'measurements': [{'elementDesc

In [7]:
# Check if objectID is already in collection
if artifacts.find_one({"objectID": item_to_add["objectID"]}):
    print("Object in artifacts already")
else:
    print("None")

Object in artifacts already


In [8]:
# Check if objectID not found
if not artifacts.find_one({"objectID": item_to_add["objectID"]}):
    # Insert the new data into the collection
    artifacts.insert_one(item_to_add)

    # Print objectID when inserted
    print(f"Adding object with objectID: {item_to_add['objectID']}")
    
else:
    print("Object in artifacts already")

Object in artifacts already


In [9]:
# Check that the new artifact was inserted
_ = artifacts.find_one({"objectID": item_to_add["objectID"]})
if _:
    pprint(_)

{'GalleryNumber': '',
 '_id': ObjectId('64f0e379a93c94ecbab7bce5'),
 'accessionNumber': '1979.206.1537',
 'accessionYear': '1979',
 'additionalImages': [],
 'artistAlphaSort': '',
 'artistBeginDate': '',
 'artistDisplayBio': '',
 'artistDisplayName': '',
 'artistEndDate': '',
 'artistGender': '',
 'artistNationality': '',
 'artistPrefix': '',
 'artistRole': '',
 'artistSuffix': '',
 'artistULAN_URL': '',
 'artistWikidata_URL': '',
 'city': '',
 'classification': 'Wood-Musical Instruments',
 'constituents': None,
 'country': 'Australia',
 'county': '',
 'creditLine': 'The Michael C. Rockefeller Memorial Collection, Bequest of '
               'Nelson A. Rockefeller, 1979',
 'culture': 'Mulga Downs Cave',
 'department': 'The Michael C. Rockefeller Wing',
 'dimensions': 'H. 20 x W.  2 3/4 x D. 5/16 in. (50.8 x 7 x 0.8 cm)',
 'dynasty': '',
 'excavation': '',
 'geographyType': '',
 'isHighlight': False,
 'isPublicDomain': False,
 'isTimelineWork': False,
 'linkResource': '',
 'locale': '',

In [10]:
# Combine the above steps to loop through the whole list of data contained in returned_data_list
# only adding to the collection when the artifact does not yet exist in the database

# Loop through returned_data_list
for _ in returned_data_list:
    # Check if the artifact already exists in the collection
    if not artifacts.find_one({"objectID": _["objectID"]}):
        # Insert the new data into the collection
        artifacts.insert_one(_)

        # Print objectID when inserted
        print(f"Adding object with objectID: {_['objectID']}")

    # Optional: Print a statement if the object is in the database.
    else:
        print("Object already in database")

Object already in database
Adding object with objectID: 313724
Adding object with objectID: 312602
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Adding object with objectID: 310384
Adding object with objectID: 317901
Adding object with objectID: 317902
Adding object with objectID: 318730
Adding object with objectID: 310343
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Adding object with objectID: 314142
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object already in database
Object alr

Data Source: [The Metropolitan Museum of Art](https://www.metmuseum.org/) (2022). The Metropolitan Museum of Art Collection API https://metmuseum.github.io/. Licensed under the [Creative Commons 0 License](https://creativecommons.org/publicdomain/zero/1.0/).<br />
Accessed Oct 3, 2022. Data collected from departmentId=5 ("Arts of Africa, Oceania, and the Americas") and search string "animal".